<a href="https://colab.research.google.com/github/kookeej/DILAB/blob/main/7.1-7.7/lab08-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab08-2 - Multi Layer Perceptron
===
하나의 퍼셉트론만으로는 ```XOR```문제를 해결할 수 없지만, ```multi layer perceptron```으로 그 문제를 해결할 수 있다. 이 ```MLP```는 ```backpropagation```을 통해 학습한다.        

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4bTVM8ErtMGl3FKHPhnGN8SyGXFhJ-HS1Ow&usqp=CAU)     
이처럼 두 개의 직선으로 XOR문제를 해결할 수 있다.



# 1. Backpropagation
입력 x가 들어왔을 때, ```Neural Network```를 통해 출력값을 구한다. 이 출력값과 실제값의 차이인 ```cost(또는 loss)```의 Weight에 대한 미분값을 통해 이 cost값을 최소화시킬 수 있도록 ```weight```를 ```update```하는 방식을 ```backpropagation```이라고 한다.     
다시 쉽게 말하면, Neural network의 결과가 잘못된 경우, 역전파를 통해 그 Error에 대한 Weight와 Bias를 수정하는 알고리즘이다.
    
하지만 많은 Layer를 가진 NN에서는 문제점이 발생한다. Layer수가 많아지게 되면, error의 의미가 점점 작아져 제대로 학습시키기가 어려워지기 때문이다.


In [1]:
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [4]:
linear1 = torch.nn.Linear(2, 2, bias=True)
linear2 = torch.nn.Linear(2, 1, bias = True)
sigmoid = torch.nn.Sigmoid()

In [5]:
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid).to(device)

In [6]:
# define cost and optimizer
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [7]:
for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0:
        print(step, cost.item())

0 0.7434073090553284
100 0.693165123462677
200 0.6931577920913696
300 0.6931517124176025
400 0.6931463479995728
500 0.6931411027908325
600 0.6931357383728027
700 0.6931295394897461
800 0.6931220889091492
900 0.6931126117706299
1000 0.6930999755859375
1100 0.693082332611084
1200 0.6930569410324097
1300 0.6930191516876221
1400 0.6929606199264526
1500 0.6928660273551941
1600 0.6927032470703125
1700 0.6923960447311401
1800 0.6917302012443542
1900 0.6899654269218445
2000 0.6838315725326538
2100 0.6561667323112488
2200 0.43110138177871704
2300 0.13489334285259247
2400 0.06630435585975647
2500 0.042168181389570236
2600 0.030453868210315704
2700 0.023665912449359894
2800 0.01927776448428631
2900 0.016224022954702377
3000 0.013983809389173985
3100 0.012273931875824928
3200 0.010928118601441383
3300 0.009842473082244396
3400 0.008949032984673977
3500 0.008201321586966515
3600 0.007566752843558788
3700 0.007021686062216759
3800 0.006548595614731312
3900 0.006134253926575184
4000 0.005768344737589

* cost값이 정상적으로 잘 줄어들고 있다.

In [8]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis:', hypothesis.detach().cpu().numpy(), '\nCorrect:', predicted.detach().cpu().numpy(), '\nAccuracy:', accuracy.item())


Hypothesis: [[0.00106364]
 [0.99889404]
 [0.99889404]
 [0.00165861]] 
Correct: [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy: 1.0


* MLP를 통해 정확도가 눈에 띄게 향상된 것을 확인할 수 있다.